In [1]:
import os
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from requests.auth import HTTPBasicAuth
import requests
import pandas as pd
from termcolor import colored
import time
import openpyxl

load_dotenv()

# Obtenha as variáveis de ambiente
username_laley = os.getenv("ELOQUA_USERNAME_LALEY")
password_laley = os.getenv("ELOQUA_PASSWORD_LALEY")
username = os.getenv("ELOQUA_USERNAME")
password = os.getenv("ELOQUA_PASSWORD")

# Agora, vamos ler o arquivo Excel "unique_emails_in_uri.xlsx"
excel_file = "PlamForm.xlsx"
baseFrame = pd.read_excel(excel_file)

# Inicialize o driver do navegador
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')  # Necessário no Windows

driver = webdriver.Chrome(options=options)

# Inicialize um dicionário para armazenar os erros agrupados por nome de formulário
erros_por_formulario = {}

# Antes do loop for
urls_com_erros = []

# Index do dataframe verificado
index = 1

for url in baseFrame['URL']:
    # Inicialize o driver do navegador
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    # options.add_argument('--disable-gpu')  # Necessário no Windows

    driver = webdriver.Chrome(options=options)
    time.sleep(2)

    url_complete = (f"{url}?cid=testethomson")
    driver.get(url_complete)

    # Pegue o valor do data-form-name usando xpath
    data_form_name = driver.find_element(
        "xpath", "//div[@data-form-name]").get_attribute("data-form-name") 

    elqSiteId = driver.find_element("id", "elqSiteId").get_attribute("value")
    print("\n" + "=" * 50 + ">" + "\n")

    print(colored(f"Form: {data_form_name}" + "\n" +
          f"Instância: {elqSiteId}" + "\n", "green"))

    # Inicialize a lista save_field
    save_field = []
    # Preencha os campos do formulário
    fields = ["firstName", "lastName", "emailAddress", "emailAddres", "emailAddressCorp", "cnpj", "Phone", "JobFunction", "jobFunction", "jobfunction1", "JobFunction1", "jobFunction1", "dropdownMenu", "dropdownMenu1", "dropdownMenu2", "dropdownMenu3", "dropdownMenu4", "company", "company", "ar_telefono", "busPhone", "mobilePhone", "zipPostal", "taxNumber1",
              "taxNumber2", "cargo1", "aRJobHierarchy1", "ar_cargo", "jobTitle", "companySize1", "companySize", "companyProfile", "companyProfile1", "companyprofile", "aRIndustry1", "practiceAreas", "stateProv", "bRState1", "country", "emailOptIn1", "optinbox", "aceite", "sou_cliente", "radioButtons", "formOptin1", "city", "id_cidade", "mensagem", "bairro"]

    for field in fields:
        try:
            element = driver.find_element("name", field)
            # Verifique se o campo está visível no formulário
            if element.is_displayed():
                if field == "emailAddress" or field == "emailAddres" or field == "emailAddressCorp":
                    value = f"teste{index}@tr.com"
                    element.send_keys(value)
                    save_field.append((field, value))
                elif field == "Phone" or field == "busPhone" or field == "mobilePhone" or field == "ar_telefono" or field == "cnpj":
                    value = "99999999999"
                    element.send_keys(value)
                    save_field.append((field, value))
                elif field == "firstName":
                    element.send_keys(url_complete)
                    save_field.append((field, value))
                elif field == "zipPostal" or field == "taxNumber1" or field == "taxNumber2":
                    value = "123456"
                    element.send_keys(value)
                    save_field.append((field, value))
                elif field == "emailOptIn1" or field == "optinbox":
                    email_opt_in_button = driver.find_element("name", field)
                    driver.execute_script(
                        "arguments[0].click();", email_opt_in_button)
                    save_field.append((field, "clicked"))
                elif field == "firstName" or field == "lastName" or field == "JobFunction" or field == "jobFunction" or field == "jobFunction1" or field == "JobFunction1" or field == "cargo1" or field == "ar_cargo" or field == "jobTitle" or field == "dropdownMenu" or field == "dropdownMenu1" or field == "dropdownMenu2" or field == "dropdownMenu3" or field == "dropdownMenu4" or field == "company" or field == "companyprofile" or field == "stateProv" or field == "country" or field == "jobfunction1" or field == "companySize" or field == "companySize1" or "companyProfile" or field == "companyProfile1" or field == "city" or field == "practiceAreas":
                    value = "Te"
                    element.send_keys(value)
                    time.sleep(1)
                    element.send_keys("e")
                    time.sleep(1)
                    element.send_keys("m")
                    time.sleep(1)
                    element.send_keys("a")
                    time.sleep(1)
                    element.send_keys("1")
                    save_field.append((field, value))
        except Exception as e:
            continue

    index += 1  # Incrementando o índice pmara a próxima URL na planilha

    submit_button = driver.find_element(
        "css selector", 'button[type="submit"]')
    driver.execute_script("arguments[0].click();", submit_button)
    # Agora a lista save_field contém os campos preenchidos e seus valores
    print(save_field)

    WebDriverWait(driver, 10).until(EC.url_changes)

    if elqSiteId == "570777387":
        base_url = "https://secure.p03.eloqua.com"
        auth = HTTPBasicAuth(
            r"laleysaeimpresora\taylor.teixeira", "@Rewmond222")
    else:
        base_url = "https://secure.p04.eloqua.com"
        auth = HTTPBasicAuth(username, password)

    # Faça a primeira chamada à API para pegar o ID do form
    api_url_1 = f"{base_url}/api/REST/1.0/assets/forms?search=name={data_form_name}"
    response = requests.get(api_url_1, auth=auth)
    if response.status_code == 200:
        data = response.json()
        if data["elements"]:
            form_id = data["elements"][0]["id"]
    else:
        print(f"Erro ao buscar ID do form:{response.status_code}")

    # Faça a chamada à API para pegar os detalhes do formulário
    api_url_2 = f"{base_url}/api/REST/2.0/assets/form/{form_id}"
    response = requests.get(api_url_2, auth=auth)
    if response.status_code == 200:
        form_details = response.json()
    else:
        print("Erro ao buscar detalhes do formulário.")

    # Crie uma lista para armazenar os pares id, name e htmlName
    elements = []
    for element in form_details.get("elements", []):
        if "id" in element and "name" in element and "htmlName" in element:
            elements.append(
                (element["id"], element["name"], element["htmlName"]))

    api_html_names = set(element[2] for element in elements)

    # Verifique se há campos em save_field que não estão na API Eloqua
    for field, value in save_field:
        if field not in api_html_names:
            # Adicionando uma condição para omitir a exibição do print para 'jobFunction'
            if field != "JobFunction" and field != "jobFunction":
                key = (data_form_name, elqSiteId)
                if key not in erros_por_formulario:
                    erros_por_formulario[key] = []
                    urls_com_erros.append(url)
                erros_por_formulario[key].append(
                    f"Campo '{field}' do formulário não foi encontrado na API Eloqua.")

    # Crie um dicionário para mapear ids para names
    id_name_mapping = {element[0]: element[1] for element in elements}

    # Faça a chamada à API para pegar os dados do formulário
    api_url_3 = f"{base_url}/api/REST/2.0/data/form/{form_id}"
    response = requests.get(api_url_3, auth=auth)
    response.raise_for_status()

    if "elements" in response.json():
        form_data_elements = (
            element for element in response.json()["elements"] if element["type"] == "FormData")
        first_form_data_element = next(form_data_elements, None)

        if first_form_data_element:
            for field_data in first_form_data_element["fieldValues"]:
                field_id = field_data.get("id", None)
                field_value = field_data.get("value", None)

                field_name = id_name_mapping.get(field_id, None)

                # Checar se field_value não é None
                if field_value:
                    if field_name == "Hidden SFDC Tracking Code" or field_name == "Hidden Media Tracking Code":
                        if field_value != "testethomson":
                            key = (data_form_name, elqSiteId)
                            if key not in erros_por_formulario:
                                erros_por_formulario[key] = []
                            erros_por_formulario[key].append(
                                f"{field_name}: {field_value}")
                        else:
                            print(
                                colored(f"{field_name}: {field_value}", 'blue'))
                    elif field_name == "Last SFDC Campaign ID" or field_name == "Hidden UniqueID":
                        print(colored(f"{field_name}: {field_value}", 'blue'))
                    else:
                        print(f"{field_name}: {field_value}")
                else:
                    # Se o campo Hidden UniqueID ou Last SFDC Campaign ID estiver ausente, adicione à lista de erros
                    if field_name == "Hidden UniqueID" or field_name == "Last SFDC Campaign ID":
                        key = (data_form_name, elqSiteId)
                        if key not in erros_por_formulario:
                            erros_por_formulario[key] = []
                        erros_por_formulario[key].append(
                            f"{field_name}: Não encontrado")

        else:
            print("Nenhum elemento do tipo 'FormData' encontrado.")
    else:
        print("A chave 'elements' não foi encontrada na resposta.")

# Feche o navegador
driver.quit()

# Crie um novo arquivo Excel
excel_output_file = "erros_formularios.xlsx"
workbook = openpyxl.Workbook()
sheet = workbook.active

# Adicione cabeçalhos às colunas
sheet['A1'] = 'Nome do Formulário'
sheet['B1'] = 'Instância'
sheet['C1'] = 'Erros'

# Preencha o Excel com os dados de erro
for idx, ((form_name, instance), errors) in enumerate(erros_por_formulario.items(), start=2):
    sheet[f'A{idx}'] = form_name
    sheet[f'B{idx}'] = instance
    sheet[f'C{idx}'] = ', '.join(errors)

# Salve o arquivo Excel
workbook.save(excel_output_file)

KeyboardInterrupt: 